### Importación de datos



In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


#1. Análisis de facturación



In [2]:
for df in [tienda, tienda2, tienda3, tienda4]:
    df["Fecha de Compra"] = pd.to_datetime(df["Fecha de Compra"], errors="coerce", dayfirst=True)
    df["año_mes"] = df["Fecha de Compra"].dt.to_period("M")

In [4]:
from numpy.random.mtrand import f
tiendas = {
    "1": tienda,
    "2": tienda2,
    "3": tienda3,
    "4": tienda4
}

opcion = input("Ingrese la tienda que desea consultar")
if opcion not in tiendas:
  print("Opción no válida")
else:
  df = tiendas[opcion]
  print(f"Resultados para la tienda {opcion}")

  filtrar = input("Desea filtrar por año y mes? (s/n)")
  if filtrar == "s":
    df["Fecha de Compra"] = pd.to_datetime(df["Fecha de Compra"])
    año_disponible = sorted(df["Fecha de Compra"].dt.year.unique())
    año_min = año_disponible[0]
    año_max = año_disponible[-1]
    print(f"Años disponibles: {año_min} - {año_max}")

    año = input("Ingrese el año")
    mes = input("Ingrese el mes")

    mes = mes.zfill(2)
    periodo = f"{año}-{mes}"
    periodo = pd.Period(periodo, freq="M")

    df_filtrado = df[df["año_mes"] == periodo]

    if df_filtrado.empty:
      print(f"No se encontraron datos para el periodo {periodo}")
    else:
      print(f"Resultados para el periodo {periodo}")

      unidades = df_filtrado["Producto"].count()
      print(f"Unidades vendidas en {periodo}: {unidades}")

      monto = df_filtrado["Precio"].sum()
      print(f"Monto total vendido en {periodo}: {monto:,.2f}")

  else:
    unidades_mes = df.groupby("año_mes")["Producto"].count()
    print("Unidades vendidas por mes:")
    print(unidades_mes)

    monto_mes = df.groupby("año_mes")["Precio"].sum()
    print("Monto total vendido por mes:")
    for periodo, monto in monto_mes.items():
      print(f"{periodo}:{monto_mes:,.2f}")



Ingrese la tienda que desea consultar2
Resultados para la tienda 2
Desea filtrar por año y mes? (s/n)s
Años disponibles: 2020 - 2023
Ingrese el año2021
Ingrese el mes4
Resultados para el periodo 2021-04
Unidades vendidas en 2021-04: 65
Monto total vendido en 2021-04: 32,978,600.00


# 2. Ventas por categoría

In [6]:
categorias = sorted(df["Categoría del Producto"].dropna().unique())

print("\nCategorías disponibles:")
for i, cat in enumerate(categorias, 1):
    print(f"{i}. {cat}")

cat_input = input("\nIngrese la categoría (tal como aparece): ")

if cat_input not in categorias:
    print("Categoría no válida")
else:
    df_cat = df[df["Categoría del Producto"] == cat_input]

    detalle = input("¿Desea filtrar detalle por año y mes? (s/n): ").lower()

    if detalle == "s":
        años = sorted(df_cat["Fecha de Compra"].dt.year.unique())
        print(f"Años disponibles: {años[0]} - {años[-1]}")

        año = int(input("Ingrese el año deseado: "))

        df_año = df_cat[df_cat["Fecha de Compra"].dt.year == año]

        if df_año.empty:
            print("No hay datos para ese año")
        else:
            ventas_mes = (
                df_año
                .groupby("año_mes")
                .agg(
                    unidades=("Producto", "count"),
                    monto=("Precio", "sum")
                )
            )

            print(f"\nVentas de '{cat_input}' en {año}:")
            print(ventas_mes)

    else:
        ventas_general = (
            df_cat
            .groupby("año_mes")
            .agg(
                unidades=("Producto", "count"),
                monto=("Precio", "sum")
            )
        )

        print(f"\nVentas históricas de '{cat_input}':")
        print(ventas_general)



Categorías disponibles:
1. Artículos para el hogar
2. Deportes y diversión
3. Electrodomésticos
4. Electrónicos
5. Instrumentos musicales
6. Juguetes
7. Libros
8. Muebles

Ingrese la categoría (tal como aparece): Libros
¿Desea filtrar detalle por año y mes? (s/n): s
Años disponibles: 2020 - 2023
Ingrese el año deseado: 2021

Ventas de 'Libros' en 2021:
         unidades     monto
año_mes                    
2021-01        13  661400.0
2021-02         8  379900.0
2021-03         3  145400.0
2021-04         3  134500.0
2021-05         6  355400.0
2021-06         4  182500.0
2021-07         1   66900.0
2021-08         3  176400.0
2021-09         2   65900.0
2021-10         5  262000.0
2021-11         7  356700.0
2021-12         5  304300.0


In [7]:
df["Año"] = df["Fecha de Compra"].dt.year
df["Q"] = df["Fecha de Compra"].dt.to_period("Q")

In [8]:
analisis_q = (
    df
    .groupby(["Categoría del Producto", "Año", "Q"])
    .agg(
        unidades=("Producto", "count"),
        monto=("Precio", "sum")
    )
    .reset_index()
)

In [11]:
df_filtros = analisis_q.copy()
filtros_usados = []

# ===== FILTRO CATEGORÍA =====
f_cat = input("¿Desea filtrar categoría? (s/n): ").lower()
if f_cat == "s":
    categorias = sorted(df_filtros["Categoría del Producto"].unique())
    print("\nCategorías disponibles:")
    for i, c in enumerate(categorias, 1):
        print(f"{i}. {c}")

    cat = input("Ingrese la categoría: ")
    if cat in categorias:
        df_filtros = df_filtros[df_filtros["Categoría del Producto"] == cat]
        filtros_usados.append(f"Categoría = {cat}")
    else:
        print("Categoría no válida")

# ===== FILTRO AÑO =====
f_año = input("¿Desea filtrar año? (s/n): ").lower()
if f_año == "s":
    años = sorted(int(a) for a in df_filtros["Año"].unique())
    print("Años disponibles:", ", ".join(map(str, años)))

    año = int(input("Ingrese el año: "))
    if año in años:
        df_filtros = df_filtros[df_filtros["Año"] == año]
        filtros_usados.append(f"Año = {año}")
    else:
        print("Año no válido")

# ===== FILTRO QUARTER =====
f_q = input("¿Desea filtrar Q? (s/n): ").lower()
if f_q == "s":
    q = input("Ingrese el Quarter (Q1, Q2, Q3 o Q4): ").upper()

    if q in ["Q1", "Q2", "Q3", "Q4"]:
        df_filtros = df_filtros[
            df_filtros["Q"].astype(str).str.endswith(q)
        ]
        filtros_usados.append(f"Q = {q}")
    else:
        print("Quarter no válido")

print("\nFiltros aplicados:")
if filtros_usados:
    for f in filtros_usados:
        print(f"- {f}")
else:
    print("Ninguno")

print("\nResultados del análisis:")
print(df_filtros)


¿Desea filtrar categoría? (s/n): s

Categorías disponibles:
1. Artículos para el hogar
2. Deportes y diversión
3. Electrodomésticos
4. Electrónicos
5. Instrumentos musicales
6. Juguetes
7. Libros
8. Muebles
Ingrese la categoría: Libros
¿Desea filtrar año? (s/n): s
Años disponibles: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
Ingrese el año: 2021
¿Desea filtrar Q? (s/n): S
Ingrese el Quarter (Q1, Q2, Q3 o Q4): Q3

Filtros aplicados:
- Categoría = Libros
- Año = 2021
- Q = Q3

Resultados del análisis:
   Categoría del Producto   Año       Q  unidades     monto
84                 Libros  2021  2021Q3         6  309200.0


# 3. Calificación promedio de la tienda


# 4. Productos más y menos vendidos

# 5. Envío promedio por tienda